# Functioning example for the ZIPRA library
This library provides tools to extract and process the .SAFE file containing Sentinel-2 data

In [1]:
import pandas as pd
from shapely import geometry
from osgeo import ogr, osr
import requests

## How to get the data

For using the library, a fundamental step is to download the zip folder containing the data. 
Below is a script that guides the user to obtain such data using the Odata endpoint provided by Copernicus Data Space Catalogue [Odata info](https://documentation.dataspace.copernicus.eu/APIs/OData.html)

In [4]:
# This section allows to define some parameters for searching Sentinel-2 products, 
# if the user already has the ID of the product, they can skip this and the following cell

# Copernicus Data Space Catalogue OData endpoint
catalogue_odata_url = "https://catalogue.dataspace.copernicus.eu/odata/v1"
collection_name = "SENTINEL-2"
product_type = "S2MSI2A"
x = ogr.Geometry(ogr.wkbPoint)
y = ogr.Geometry(ogr.wkbPoint)
source_srs = osr.SpatialReference()

# USER DEFINED PARAMETERS
# Define the maximum cloud coverage
max_cloud_cover = 30

# An easy way of getting the coordinates for the BBox is by opening QGIS, adding the OSM Standard layer,
# zooming to the area of interest and positioning the cursor the coordinates will appear on the bottom.
x.SetPoint(0, 1030653, 5689453) #top left corner
y.SetPoint(0, 1016885, 5700000) #bottom right corner
source_srs.ImportFromEPSG(3857)

#Define the period
search_period_start = "2025-09-15T00:00:00.000Z"
search_period_end = "2025-09-30T00:00:00.000Z"
# END OF USER DEFINED PARAMETERS

target_srs = osr.SpatialReference()
target_srs.ImportFromEPSG(4326)
coord_transform = osr.CoordinateTransformation(source_srs, target_srs)
x.Transform(coord_transform)
y.Transform(coord_transform)
Bbox=[x.GetX(), x.GetY(), y.GetX(), y.GetY()] 
aoi = geometry.box(*Bbox).wkt

In [3]:
search_query = f"{catalogue_odata_url}/Products?$filter=Collection/Name eq '{collection_name}' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{product_type}') and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}') and ContentDate/Start gt {search_period_start} and ContentDate/Start lt {search_period_end} and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {max_cloud_cover})"
response = requests.get(search_query).json()
result = pd.DataFrame.from_dict(response["value"])
result.head(5)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,3d8f051c-8a62-4b51-be25-53a5d16092c4,S2C_MSIL2A_20250928T071711_N0511_R006_T38PNR_2...,application/octet-stream,1195125233,2025-09-28T13:47:58.000000Z,2025-09-28T13:55:21.002589Z,2025-09-28T13:55:21.002589Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/28/S2C_MSIL...,"[{'Value': 'ec41ed5cca7f050ca959e3b4be165e02',...","{'Start': '2025-09-28T07:17:11.025000Z', 'End'...",geography'SRID=4326;POLYGON ((45.9999346860839...,"{'type': 'Polygon', 'coordinates': [[[45.99993..."
1,application/octet-stream,13c3bc5c-69ed-4427-a340-297556539853,S2C_MSIL2A_20250918T071631_N0511_R006_T38PNR_2...,application/octet-stream,1179669882,2025-09-18T13:47:51.000000Z,2025-09-18T13:52:41.545280Z,2025-09-18T13:52:41.545280Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/18/S2C_MSIL...,"[{'Value': '6608223d1779ffd9d0bfffa046607282',...","{'Start': '2025-09-18T07:16:31.025000Z', 'End'...",geography'SRID=4326;POLYGON ((45.9997800779024...,"{'type': 'Polygon', 'coordinates': [[[45.99978..."
2,application/octet-stream,f9da8b38-43d2-4c18-82cc-9f9f17aac57e,S2B_MSIL2A_20250923T071619_N0511_R006_T38PNR_2...,application/octet-stream,1175296376,2025-09-23T10:45:01.000000Z,2025-09-23T10:49:29.693814Z,2025-09-23T10:49:29.693814Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/23/S2B_MSIL...,"[{'Value': '0632eaa51872f75dee861f30ec61c4a9',...","{'Start': '2025-09-23T07:16:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((45.9998268394936...,"{'type': 'Polygon', 'coordinates': [[[45.99982..."


In [4]:
# Select the image you want to download

# USER DEFINED PARAMETERS
Index_image = 0  # Change this index to select different images from the search result
path=f"./DATA/"
# END OF USER DEFINED PARAMETERS

Image_Id = result.iloc[Index_image]["Id"]
Image_name = result.iloc[Index_image]["Name"]
url = f"https://download.dataspace.copernicus.eu/odata/v1/Products({Image_Id})/$value"
path = path + f"{Image_name}.zip"

Before proceding with the download, the user should create an account on Copernicus Data Space and get OAuth client, a guide can be found at this [link](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html)

In [5]:
# Authentication credentials
# USER DEFINED PARAMETERS
username = "sh-775c3bd5-96e1-40c2-8d16-d98131ab32ba"
password = "K2U3wVQLSNshUfZfk82MggSDdbKP7mIr"
# END OF USER DEFINED PARAMETERS

In [9]:
# Get an access token and establish a session
token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
token_data = {
    "grant_type": "client_credentials",
    "client_id": username,
    "client_secret": password
}
token_response = requests.post(token_url, data=token_data)
access_token = token_response.json().get("access_token")

print("Token response:", token_response.status_code)
session = requests.Session()
session.headers["Authorization"] = f"Bearer {access_token}"

Token response: 200


In [ ]:
#Download the image, this might take few minutes
try:
    response = session.get(url, stream=True)
    response.raise_for_status()
    zip_data = response.content
    response.close()

    # Writes the downloaded content to a file
    with open(path, 'wb') as f:
        f.write(zip_data)

    print(f"\nData saved to '{path}' (Size: {len(zip_data)} bytes).")

except requests.exceptions.RequestException as e:
    print(f"\nAn error occurred during the request: {e}")

## Zipra library example

In [6]:
from ZIPRA import Band_estraction
#path="DATA/S2B_MSIL2A_20250923T071619_N0511_R006_T38PNR_20250923T100114.SAFE"
tiff_file, band_list = Band_estraction(path)

File ZIP decompressed successfully.
band folder: ./DATA/S2C_MSIL2A_20250928T071711_N0511_R006_T38PNR_20250928T125215.SAFE\GRANULE\L2A_T38PNR_A005553_20250928T073402\IMG_DATA
Band B02 found at ./DATA/S2C_MSIL2A_20250928T071711_N0511_R006_T38PNR_20250928T125215.SAFE\GRANULE\L2A_T38PNR_A005553_20250928T073402\IMG_DATA\R10m\T38PNR_20250928T071711_B02_10m.jp2
Band B03 found at ./DATA/S2C_MSIL2A_20250928T071711_N0511_R006_T38PNR_20250928T125215.SAFE\GRANULE\L2A_T38PNR_A005553_20250928T073402\IMG_DATA\R10m\T38PNR_20250928T071711_B03_10m.jp2
Band B04 found at ./DATA/S2C_MSIL2A_20250928T071711_N0511_R006_T38PNR_20250928T125215.SAFE\GRANULE\L2A_T38PNR_A005553_20250928T073402\IMG_DATA\R10m\T38PNR_20250928T071711_B04_10m.jp2
Band B08 found at ./DATA/S2C_MSIL2A_20250928T071711_N0511_R006_T38PNR_20250928T125215.SAFE\GRANULE\L2A_T38PNR_A005553_20250928T073402\IMG_DATA\R10m\T38PNR_20250928T071711_B08_10m.jp2
Band B12 found at ./DATA/S2C_MSIL2A_20250928T071711_N0511_R006_T38PNR_20250928T125215.SAFE\GRA

In [15]:
print("Band_list:", band_list)
print("Tiff_file:", tiff_file)

Band_list: ['B02', 'B03', 'B04', 'B08', 'B12', 'SCL']
Tiff_file: ./DATA\Bands_extracted.tif


Area calculations:

In [14]:
from ZIPRA import Area_calculation, Clip_AOI
import rasterio

tiff_file="./DATA/Bands_extracted.tif"
class_list = [1,2,3]
Area = Area_calculation(tiff_file, class_list)
print("Total area of specified classes (in m^2):", Area)

Total area of specified classes (in m^2): 32000.0


In [13]:
from rasterio import mask
from shapely import wkt

with rasterio.open(tiff_file) as src:
    print("CRS of the raster:", src.crs)
    
x_clip = ogr.Geometry(ogr.wkbPoint)
y_clip = ogr.Geometry(ogr.wkbPoint)

x_clip.SetPoint(0, 1020327, 5692085) #top left corner
y_clip.SetPoint(0, 1027209, 5697368) #bottom right corner

x_clip.Transform(coord_transform)
y_clip.Transform(coord_transform)
Bbox_clip=[x_clip.GetX(), x_clip.GetY(), y_clip.GetX(), y_clip.GetY()] 
aoi_clip = geometry.box(*Bbox_clip).wkt

clipped_tiff = Clip_AOI(tiff_file, aoi_clip)

CRS of the raster: EPSG:32638


ValueError: Input shapes do not overlap raster.